In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import json

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("IMDB Dataset.csv")
print (df.head())

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))

In [ ]:
def preprocess(text):
    text = re.sub(r'<.*?>', '', text)
    tokens = word_tokenize(text.lower())
    lemmatizer = WordNetLemmatizer()
    processed_tokens = []
    negation = False
    for word in tokens:
        if word in ['not', 'never']:
            negation = True
        elif word.isalpha() and word not in stop_words:
            if negation:
                word = 'not_' + word  
                negation = False
            word = lemmatizer.lemmatize(word)  
            processed_tokens.append(word)
        else:
            negation = False
    return ' '.join(processed_tokens)

print('\n\nStarting Preprocessing')
df['review'] = df['review'].apply(preprocess)
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})
print('\n\nFinished Preprocessing')

In [ ]:
print(df.head())

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], random_state = 0, test_size=0.15)

In [ ]:
'''
this cell is very computationally taxing due to it running 720 combinations 5 times each, totalling in 3600 fits
if you need to run it to see if it works, i'd recommend deleting the n_jobs= -1 flag as it will use all resources on your computer, making it difficult to stop execution
'''

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(min_df=5)),
    ('lr', LogisticRegression(max_iter=1000, solver='saga'))
])

param_grid = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4)],
    'tfidf__max_features': [5000, 10000, None],
    'lr__C': [0.01, 0.1, 1, 10, 100],
    'lr__penalty': ['l2', 'l1', 'elasticnet', None], 
    'lr__l1_ratio': [0.2, 0.5, 0.8]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1, error_score='raise')
grid_search.fit(x_train, y_train)

In [ ]:
print("Best parameters found: ", grid_search.best_params_)
best_parameters = grid_search.best_params_
best_model = grid_search.best_estimator_
predictions = best_model.predict(x_test)

accuracy = accuracy_score(y_test, predictions)
print('Optimized Model Accuracy:' accuracy)
print(classification_report(y_test, predictions))
print('AUC score:', roc_auc_score(y_test, predictions))
with open('bestparameters_LR-HP.json', 'w') as file:
    json.dump(best_parameters, file)

In [ ]:
vect = TfidfVectorizer(min_df=5, ngram_range=(1, 5))
x_train_vectorised = vect.fit_transform(x_train)
x_test_vectorised = vect.transform(x_test)

models = {
    'MultinomialNB': MultinomialNB(alpha=0.1),
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'SVC': SVC(),
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier()
}

def model_experiment(models, x_train, y_train, x_test, y_test):
    for name, model in models.items():
        model.fit(x_train, y_train)
        predictions = model.predict(x_test)
        accuracy = accuracy_score(y_test, predictions)
        print(name, 'Accuracy:', accuracy)
        print(classification_report(y_test, predictions))
        print('AUC score:', roc_auc_score(y_test, predictions))
        print('\n')

model_experiment(models, x_train_vectorised, y_train, x_test_vectorised, y_test)

In [ ]:
vect = TfidfVectorizer(min_df=5, ngram_range=(1, 4))
x_train_vectorised = vect.fit_transform(x_train)

model = LogisticRegression(C=10, penalty='l2', max_iter=1000) 

model.fit(x_train_vectorised, y_train)
x_test_vectorised = vect.transform(x_test)
predictions = model.predict(x_test_vectorised)

print('\n\n')
accuracy = accuracy_score(y_test, predictions)
print('Accuracy:', accuracy)
print(classification_report(y_test, predictions))
print('AUC score is: ', roc_auc_score(y_test, predictions))

In [ ]:
cm = confusion_matrix(y_test, predictions)
# Plotting using Seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Purples', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(x_test_vectorised)[:,1])
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
report = classification_report(y_test, predictions, output_dict=True)
sns.heatmap(pd.DataFrame(report).iloc[:-1, :].T, annot=True)